# Reusing Results

This notebook demonstrates how to reuse previously computed results across tools and across analyzers.

---

Include the repository root in your path and import all required packages and components from the Model Agnostic Toolkit.

In [17]:
import sys
sys.path.insert(0, '..')

from model_agnostic_toolkit import Model, ImportanceAnalyzer, InteractionAnalyzer
from model_agnostic_toolkit.datasets import MultiplicationTwoDataset
from model_agnostic_toolkit.tools import PDPImportance, PDPInteraction, VIPImportance, VIPInteraction, ALEImportance, ALEInteraction

Create or load a dataset and retrieve its features and feature pairs. Additionally, create the default gradient-boosted tree model.

In [18]:
data = MultiplicationTwoDataset(file='../data/multiplication_two.hdf5')
features = data.features
feature_pairs = data.feature_pairs

Loading data from ../data/multiplication_two.hdf5 ... done.


In [19]:
model = Model(data_type=data.data_type)

Created XGBRegressor.


Create an importance analyzer with a list of tool instances you want to run and start the analysis by calling its `run_analysis()` method.

Without needing to do more, results of these tools are already reused by the respective other tools. Here, the VIP importance tool internally relies on PDP importance results and reuses the existing ones without recomputation. The order of tools does not matter.

In [20]:
tools_importance = [PDPImportance(), VIPImportance()]
ana_importance = ImportanceAnalyzer(model=model, dataset=data, tools=tools_importance)

res_importance = ana_importance.run_analysis(features=features)

In addition to sharing results across tools within an analyzer, results can also be shared across analyzers.

Create an interaction analyzer with a list of tool instances you want to run. Pass the previous importance analyzer with the `reuse_analyzers` parameter. Then start the analysis.

The interaction analyzer can then reuse the importance analyzer's results.

In [21]:
tools_interaction = [PDPInteraction(), VIPInteraction()]
ana_interaction = InteractionAnalyzer(model=model, dataset=data, tools=tools_interaction,
                                  reuse_analyzers=ana_importance)

res_interaction = ana_interaction.run_analysis(features=feature_pairs)

Tools can also be reused if they have not been run before. Results from running them through other tools are then temporarily stored in their analyzer until they are evaluated themselves. Also, reuse analyzers can be added even after an analyzer is initialized.

Create an importance analyzer with a list of tool instances you want to run.

In [22]:
tools_importance = [ALEImportance()]
ana_importance = ImportanceAnalyzer(model=model, dataset=data, tools=tools_importance)

Add tools to the interaction analyzer and specify the importance analyzer as a reuse analyzer via the `add_reuse_analyzers()` method. Then run the interaction analysis again (this yields all tools' results, including those computed earlier).

Here, the ALE interaction internally relies on ALE importance results. Those are computed and temporarily stored in the importance analyzer.

In [23]:
ana_interaction.add_tools(ALEInteraction(include_first_order=True))
ana_interaction.add_reuse_analyzers(ana_importance)

res_interaction = ana_interaction.run_analysis(features=feature_pairs)

Start the importance analysis. The temporary results for ALE importance, computed while calculating the ALE interaction, are now reused and not recomputed.

In [24]:
res_importance = ana_importance.run_analysis(features=features)